In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers==2.9.0
!nvidia-smi

Fri May 13 21:42:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
set_seed(42)

In [4]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('drive/MyDrive/outputs_new/model_files')

In [5]:
# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in t5_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in t5_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=1e-8)

In [6]:
true_false_adjective_tuples = [
                               (("Can you lend me some money?",	"I'm completely broke until payday."), "no"), 
                               (("Do you think he did the theft?",	"He is as innocent as a lamb"), "no"),
                               (("Are you sure you want the spoiler",	"haha sure. by the time, I get to it; I will forget the details, only knowing that it's touching at the end"),"yes"),
                               (("Does tomorrow afternoon work for you?",	"Yeah tomorrow afternoon works for me."),	"yes"),
                               (("oh wow, Cuban cigar? I've never smoke before",	"This one is actually from Peru, which is more my taste."),	"no"),
                               (("Are you sending them an invitation?",	"Certainly."),	"yes"),
                               (("You, uh, you don't want to go for a ride, do you?",	"Is it safe?"),	'yes'),
                               (("Have you ever made them yourself?",	"no. only eat them"),	"no"),
                               (("Have you seen my watch?",	"I will take a look for it around my house."),	"no"),
                               (("You don't look normal. Are you all right?", "I have a headache."),	"no"),
                               (("any improvements in your marathon time?",	"No, only a few minutes. I still find it difficult to run long races"),	"no"),
                               (("do you live near San Mateo?",	"no, but I'm willing to drive for good ramen"),	"no"),
                               (("Can he play the violin?",	"Can a pig fly?"),	"no"),
                               (("Aren't you scared?", "Sometimes."),	"yes")
]

In [7]:
t5_model.train()

epochs = 10

for epoch in range(epochs):
  print ("epoch ",epoch)
  for input,output in true_false_adjective_tuples:
    input_sent = "Determine the implicature: "+input[0] + input[1]+ " </s>"
    ouput_sent = output+" </s>"

    tokenized_inp = tokenizer.encode_plus(input_sent,  max_length=96, pad_to_max_length=True,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(ouput_sent, max_length=96, pad_to_max_length=True,return_tensors="pt")


    input_ids  = tokenized_inp["input_ids"]
    attention_mask = tokenized_inp["attention_mask"]

    lm_labels= tokenized_output["input_ids"]
    decoder_attention_mask=  tokenized_output["attention_mask"]


    # the forward function automatically creates the correct decoder_input_ids
    output = t5_model(input_ids=input_ids, lm_labels=lm_labels,decoder_attention_mask=decoder_attention_mask,attention_mask=attention_mask)
    loss = output[0]

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

epoch  0


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9


In [8]:
import csv
file = csv.reader(open('testing.csv'))
correct = 0
wrong = 0
total = 0
tp = 0
tn = 0
fp = 0
fn = 0
for line in file:
  test_sent = "Determine the implicature: " + line[0] + "</s>"
  test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

  test_input_ids  = test_tokenized["input_ids"]
  test_attention_mask = test_tokenized["attention_mask"]

  t5_model.eval()
  beam_outputs = t5_model.generate(
      input_ids=test_input_ids,attention_mask=test_attention_mask,
      max_length=64,
      early_stopping=True,
      num_beams=10,
      num_return_sequences=1,
      no_repeat_ngram_size=2
  )

  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      print (sent, line[1])
      if sent == line[1]:
        correct += 1
      else:
        wrong += 1
      if sent == 'yes' and line[1] == 'yes':
        tp += 1
      elif sent == 'yes' and line[1] == 'no':
        fp += 1
      elif sent == 'no' and line[1] == 'yes':
        fn += 1
      elif sent == 'no' and line[1] == 'no':
        tn += 1
      print(correct, wrong)
  print("")
  total += 1
  print(total)

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


no no
1 0

1
yes no
1 1

2
no no
2 1

3
yes no
2 2

4
yes no
2 3

5
yes yes
3 3

6
yes yes
4 3

7
yes no
4 4

8
yes yes
5 4

9
yes yes
6 4

10
yes yes
7 4

11
yes no
7 5

12
yes no
7 6

13
no no
8 6

14
yes no
8 7

15
yes yes
9 7

16
yes yes
10 7

17
yes yes
11 7

18
yes yes
12 7

19
yes yes
13 7

20
yes yes
14 7

21
yes yes
15 7

22
no no
16 7

23
yes yes
17 7

24
yes no
17 8

25
yes yes
18 8

26
no no
19 8

27
yes yes
20 8

28
yes yes
21 8

29
yes yes
22 8

30
no yes
22 9

31
yes yes
23 9

32
yes yes
24 9

33
yes no
24 10

34
no yes
24 11

35
no no
25 11

36
yes yse
25 12

37
yes yes
26 12

38
yes yes
27 12

39
yes yes
28 12

40
yes no
28 13

41
no no
29 13

42
no no
30 13

43
yes no
30 14

44
no no
31 14

45
yes yes
32 14

46
yes yes
33 14

47
yes yes
34 14

48
yes no
34 15

49
yes yes
35 15

50
yes yes
36 15

51
yes yes
37 15

52
no yes
37 16

53
yes yes
38 16

54
yes yes
39 16

55
yes yes
40 16

56
yes yes
41 16

57
yes yes
42 16

58
yes no
42 17

59
yes yes
43 17

60
yes yes
44 1

In [9]:
print("Correct Predictions: ",correct)
print("Incorrect Predictions:", wrong)
print("Accuracy:", correct/total)

Correct Predictions:  134
Incorrect Predictions: 48
Accuracy: 0.7362637362637363


In [10]:
recall = tp / (tp + fn)
precision = tp / (tp + fp)
f1_score = (2 * recall * precision) / (recall + precision) 
print("Recall: ", recall)
print("Precision: ", precision)
print("F1 score: ", f1_score)

Recall:  0.9166666666666666
Precision:  0.7226277372262774
F1 score:  0.8081632653061225
